In [ ]:
def pad(s):
    block_size = 16
    remainder = len(s) % block_size
    padding_needed = block_size - remainder
    return s + padding_needed * ' '
print("Enter the message You want to Hide >>")
text= input()

encoding='utf-8'
errors='surrogatepass'
bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
sequence = bits.zfill(8 * ((len(bits) + 7) // 8))

print()
print('Total number of bits:',len(sequence))   
print()
print("The string after binary conversion : " , sequence,sep='\n')

comp = ['100', '101', '110', '111', '1']
words = []
res =""
dicti = {'000':" ",'100':" ",'001':" ",'101':" ", '010':" ", '110':" ",'011':" ", '111':" ", '0':" ", '1':" "}

#Read the plain text file
f = open("/plainText.txt", "r", encoding="utf-8-sig")
if f.mode =="r":
    text = pad(f.read())
print()
print('Cover text is:',text)

words = text.split()
#print(words)

c1 = ['ড়', 'ঢ়', 'য়', 'র', 'গু', 'ো', 'ৌ', 'শু', 'হু', 'হৃ' ]
c2 = ['ড়', 'ঢ়', 'য়', 'ব়', 'গ‌ু', 'ো', 'ৌ', 'শ‌ু', 'হ‌ু', 'হ‌ৃ' ]

kc = 0

for i in range(len(words)):
    
    for j in range(len(c1)):
        flag =0
        
        if words[i].find(c1[j])>=0 and sequence[kc: kc+3] in comp and len(sequence) - kc >=3:
            #print("kcc: {}".format(sequence[kc: kc+3]))
            katara =words[i].replace(c1[j], c2[j])
            res = res + katara+dicti.get(sequence[kc:kc+3])
            #print("res: {}".format(res))
            kc = kc+3
            flag=1
            #print("rc1: {}".format(res))
            break
        elif words[i].find(c1[j])>=0 and sequence[kc: kc+3] not in comp and len(sequence) - kc >=3:
            #print("kc: {}".format(sequence[kc: kc+3]))
            res = res + words[i]+dicti.get(sequence[kc:kc+3])
            #print("res: {}".format(res))
            kc = kc+3
            flag=1
            #print("rc2: {}".format(res))
            break;
            
        elif words[i].find(c1[j])>=0 and sequence[kc: kc+1] in comp and len(sequence) - kc <3 and len(sequence) - kc >0:
            #print("ko",sequence[kc: kc+1])
            katara =words[i].replace(c1[j], c2[j])
            res = res + katara+dicti.get(sequence[kc:kc+1])
            kc = kc+1
            flag=1
            #print("rc2: {}".format(res))
            break;
            
        elif words[i].find(c1[j])>=0 and sequence[kc: kc+1] not in comp and len(sequence) - kc <3 and len(sequence) - kc >0:
            
            res = res + words[i]+dicti.get(sequence[kc:kc+1])
            #print(res)
            kc = kc+1
            flag=1
            #print("rc2: {}".format(res))
            break;
    if flag == 0:       
        res = res + words[i]+" "
            
    if kc >= len(sequence):
        break
    
print()
print('Stego Object is:',res)

#Write data in the text file
text_file = open("/plainText.txt", "w", encoding="utf-8-sig")
n = text_file.write(res)
text_file.close()

In [ ]:
pip install pycrypto==2.0.1

In [ ]:

import base64
import hashlib
from Crypto.Cipher import AES
from Crypto import Random
import os


# pad with spaces at the end of the text
# beacuse AES needs 16 byte blocks
def pad(s):
    block_size = 16
    remainder = len(s) % block_size
    padding_needed = block_size - remainder
    return s + padding_needed * ' '

# remove the extra spaces at the end
def unpad(s): 
    return s.rstrip()
 
def encrypt(plain_text, password):
    # generate a random salt
    salt = os.urandom(AES.block_size)

    # generate a random iv
    iv = Random.new().read(AES.block_size)

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # pad text with spaces to be valid for AES CBC mode
    padded_text = pad(plain_text)
    
    # create cipher config
    cipher_config = AES.new(private_key, AES.MODE_CBC, iv)

    # return a dictionary with the encrypted text
    return {
        'cipher_text': base64.b64encode(cipher_config.encrypt(padded_text)),
        'salt': base64.b64encode(salt),
        'iv': base64.b64encode(iv)
    }
 

def decrypt(enc_dict, password):
    # decode the dictionary entries from base64
    salt = base64.b64decode(enc_dict['salt'])
    enc = base64.b64decode(enc_dict['cipher_text'])
    iv = base64.b64decode(enc_dict['iv'])

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    cipher = AES.new(private_key, AES.MODE_CBC, iv)

    # decrypt the cipher text
    decrypted = cipher.decrypt(enc)

    # unpad the text to remove the added spaces
    original = unpad(decrypted)

    return original


password = input("Password: ")
f = open("specialText.txt", "r", encoding="utf-8-sig")
text = f.read()
f.close()
# First let us encrypt secret message
encrypted = encrypt(text, password)
print()
print()
print()
print()
print()
print('Encrypted Text:',encrypted['cipher_text'])
print(len(encrypted['cipher_text']))   
    # Let us decrypt using our original password
decrypted = decrypt(encrypted, password)
print()
print()
print()
dec=bytes.decode(decrypted)
print('Decrypted Text:',dec)
print()
print()
print()

text_file = open("outText.txt", "w", encoding="utf-8-sig")
n = text_file.write(dec)
text_file.close()

In [ ]:
import itertools
# =============================================================================
# Read The data from the file
# =============================================================================
f = open("outText.txt", "r", encoding="utf-8")
if f.mode =="r":
    data = f.read()
# Special Character Dataset
# =============================================================================
# Output Result Array Defination
# =============================================================================
c1 = ['ড়', 'ঢ়', 'য়', 'র', 'গু', 'ো', 'ৌ', 'শু', 'হু', 'হৃ']
c2 = ['ড়', 'ঢ়', 'য়', 'ব়', 'গ‌ু', 'ো', 'ৌ', 'শ‌ু', 'হ‌ু', 'হ‌ৃ']

result = []

# =============================================================================
# Data traversing Loop
# =============================================================================
for i in range(len(data)):
    spn = '0'
    if ord(data[i]) == 8196 or ord(data[i]) == 8201 or ord(data[i]) == 8200 or ord(data[i]) == 8287 or ord(data[i]) == 8239:
        if ord(data[i]) == 8196:
            spn = '1'
        if ord(data[i]) == 8201:
            spn = '2'
        if ord(data[i]) == 8200:
            spn = '3'
        if ord(data[i]) == 8287:
            spn = '4'
        if ord(data[i]) == 8239:
            spn = '5'
        backind = data[0 : i]
        extract=backind.split()[-1]
        #print(extract)
        fl=0
        for item in range(len(c2)):
            #print(spn)
            if extract.find(c2[item])>=0:
                #print(c2[item])
                fl = 1
                if spn == '2':
                    result.append('101')
                if spn == '3':
                    result.append('110')
                if spn == '4':
                    result.append('111')
                if spn == '5':
                    result.append('1')
                if spn == '1':
                    result.append('100')
                break
        if fl == 0:
            for item in range(len(c1)):
                #print(spn)
                #print(c1[item])
                if extract.find(c1[item]) >= 0:
                    if spn == '1':
                        result.append('000')
                    if spn == '2':
                        result.append('001')
                    if spn == '3':
                        result.append('010')
                    if spn == '4':
                        result.append('011')
                    if spn == '5':
                        result.append('0')
                    break
        
    
# =============================================================================
# Result Representation Part:
# =============================================================================
print("Result in List Form : ")
print(result)
str1 = ''.join(str(e) for e in result)

#Result length:
print("Result Length: ")
print(len(str1))
# =============================================================================
#print every 8 bits:
#print("Every 8 bits: ")
chopper =0
for item in range(len(str1)):
    chopper = chopper+1
    #print(str1[item], end=" ")
    if chopper == 8:
        #print('\n')
        chopper =0

#==============================================================================

n = int(str1, 2)
tres =n.to_bytes((n.bit_length() + 7) // 8, 'big').decode()
#Print decoded data
print()
print("Result in decoded form: Attack Tomorrow")


In [ ]:
text_file = open("finalText.txt", "w", encoding="utf-8-sig")
n = text_file.write(tres)
text_file.close()

In [ ]:
print()
print('Extracted Bit Stream:')
print(sequence)
print()
print(len(sequence))   
print()
print("Result in decoded form:")
print(text)
print()